In [1]:
%%HTML
<link rel="stylesheet" href="https://doc.splicemachine.com/jupyter/css/custom.css">

In [ ]:
import os
os.environ['JDBC_HOST'] = 'jrtest01-splice-hregion'

# Architecture

This notebook provides an architectural overview of the architecture of Splice Machine, to expand your understanding of how our database works. We'll cover the following topics:

* *Dual Engine Architecture*
* *Hadoop Components*
* *Splice Concepts*
* *Query Execution*
* *Authentication and Authorization*



## Dual Engine Architecture

Splice Machine processes SQL queries using __both__ HBase and Spark: 

* Shorter, *transactional* queries are processed in Hbase; this is known as Online Transactional Processing, or *OLTP*. 
* Longer, *analytical* queries are processed in Spark; this is known as Online Analytical Processing, or *OLAP*.

As you can see from the following diagram, this application of two processing engines ensures that bigger jobs don't block the lanes of smaller jobs:

<img class="splice" src="https://s3.amazonaws.com/splice-training/external/images/WorkloadIsolation.png" width=800>

### HBase Queries

The HBase engine is very fast at SQL queries that involve:

* quick indexed lookups
* short scans
* small joins on smaller result sets (e.g. the NestedLoop join)

When the HBase engine is used in this way, queries are expected to return in the neighborhood of 5ms to 100ms.

### Spark Queries

When larger analytic queries such as larger scans and joins come along, you need tools such as those available in Spark to handle the job. These analytical queries may take much longer to complete: instead of milliseconds, you might see some queries taking seconds, minutes, or even hours to run.

Spark brings a host of capabilities to bear in processing these large queries, including these:

* Job/Stage/Task mechanism that structures the work into multiple steps and manages it in chunks at a time
* Multiple executors to parallelize the work where possible (especially large scans)
* Spill-to-disk mechanisms so that processing does not fail if intermediate results do not all fit into memory

You may recall having seen these in previous tutorials when using the Spark UI to visualize the progress of an analytic SQL query.

### Making the Decision

When Splice Machine builds its query plan, it assesses the plan for overall fitness to ultimately decide if it should go to HBase or to Spark.  The details of how the assessment algorithm works are subject to change, but it is currrently on whether or not the __maximum__ number of scanned rows exceeds 10,000 in the entire plan. If so, the query is sent to Spark. If not, it goes to HBase.  

As a reminder, when you run `EXPLAIN` on your query, you can see which engine will be used to process it: the top line of the plan displays either:

* `engine=Spark` if the query will be processed by Spark
* `engine=control` if the query will be processed by HBase

For example:

   ```
   Cursor(n=7,rows=1,updateMode=READ_ONLY (1),engine=Spark)
   ```


## Hadoop Components

You are already no doubt familiar with Hadoop and the benefits it provides to help solve today's large-scale problems. In this section, we'll go in a little more detail
regarding how that pertains to Splice Machine, including information about critical Hadoop components that Splice Machine relies upon.  

### ZooKeeper

ZooKeeper provides critical capabilities for services such as HBase and HDFS. Splice Machine also tapes into ZooKeeper to help it manage critical capabilities, including:

* Managing the state of the Splice Machine database cluster.
* Timestamp generation, which is an essential part of the Splice Machine transaction system.

Be sure to include a ZooKeeper Quorum for your more critical clusters; this should be an odd number (typically 3, sometimes more for larger clusters) to ensure that there are always enough ZooKeeper servers up and running to keep the service available.


### HDFS

Here are some key elements to keep in mind about the Hadoop Distributed File System (HDFS):

* Since HDFS is distributed, the entire filesystem (and indeed individual files) can be LARGER than what can fit on any one machine on your cluster; this is a very powerful capability and is critical for our large scale purposes.
* Do not confuse HDFS with any one machine's normal filesystem; you must use different commands to transfer files in or out, check available capacity, etc.
* As with the rest of Hadoop, HDFS is designed with the understanding that the individual machines may be commodity hardware and therefore may fail.  HDFS has a Replication Factor (default 3) that indicates how many copies of each piece of data are made automaticall, which provides good fault tolerance. If your particular use case calls for it, you can modify the replication factor.
* There are two types of nodes in HDFS: 1) _Name_ nodes are the workhorses and store the data, and 2) _Data_ nodes behave like Masters that manage the filesystem namespace. Here's how it looks:

  <img class="fit3qtrwidth" src="https://s3.amazonaws.com/splice-training/external/images/HDFS.png">

### More about HBase

HBase is a Key/Value store that sits on top of HDFS that the Splice Machine database uses for durable storage. There are a number of key aspects of HBase that make it well suited as an excellent backing store for Splice Machine. 

HBase:

* Was designed for extremely high performance of inserts/updates and _delete_s (see below); it has proven millisecond performance at Petabyte scale.
* Has an extensible architecture with coprocessors; this is how Splice Machine add capabilities such as its full transactionality (which HBase does not have).
* Supports further performance enhancements called _short circuit reads_ to take advantage of locality, and to not have to read everything through HDFS.
* Deletes data by inserting a _tombstone_ on the record, rather than wiping the record. This makes it possible for Splice Machine to transactionally roll back a record deletion.
* Has a notion of tables with rows and columns, which Splice Machine leverages.
* Is _auto-sharding:_ as tables of data grow with increasing rows, it is important in a distributed architecture for that data to be spread out (sharded) across the cluster, i.e. broken into _regions_.  Since this happens automatically, users do not need to deal with partitioning, which is also known as _Region Splitting_.  Note that you can manually control splitting, if so desired.
* Stores its _Regions_ on _Region Servers_.  You should keep the count of Regions per Region Server to below 200-400.
* Stores _Key/Value_ stores auto-sorted by key. In Splice Machine's case, this is designed to be the _Primary Key_ of the SQL Table being stored. 

## Splice Components

This section provides information how specific components of Splice Machine work within its architecture.

### Tables and Indexes

As previously indicated, there is a natural mapping between HBase tables and Splice Machine tables; here are some important similarities and differences between the two:

* Splice Machine tables always have a fully specified schema.  They may have nullable columns, but every row is a fully specified valid SQL row of data for that table.
* Splice Machine tables may or may not have a primary key; the key can also be composite of multiple columns.
* If a Splice Machine table has no primary key, a salt key is used on the HBase side, since HBase still needs some kind of key; note that this prevents ANY kind of quick single-row lookup.

A Splice Machine Index is just another HBase table behind the scene, with the Index definition as the Key, and the base table row reference as the Value.

### Rows and Encodings

* In Splice Machine we only use one column family, even though HBase supports multiple column families. In fact Splice Machine stores entire rows in single column. So if you look at an HBase Row directly, it will have a RowKey and a single data column.
* Row data is encoded and bit packed to preserve lexicographical sort order.

<p class="noteImportant">There is <strong>no direct API</strong> to read non-Splice HBase data using SQL.</p>


## Query Execution

Let's explore more about the _decision point_ for whether a query should run in HBase or Spark. The following image illustrates the full process (HBase or Spark), starting with receiving an SQL query, all the way through executing it on the proper engine:

<img class="fithalfwidth" src="https://s3.amazonaws.com/splice-training/external/images/QueryExecution.png" />

To review these steps:

* The SQL must be parsed, planned, and optimized, and byte code generated.  At this point, we have enough information to know whether we will be running on HBase or Spark.
* If we'll be using HBase, we have the HBase block cache and bloom filters to call upon as part of what HBase can do to maximize its performance on large-scale queries.
* If we're using Spark, we know that we are kicking off a large job, so we can leverage information directly from HFiles and Memstore for performance improvements along the way.

## Where to Go Next
The next notebook in this class, [*Statistics*](./c.%20Statistics.ipynb), shows you how we use database statistics to optimize queries, and how you can use those same statistics for query tuning.
